In [ ]:
import pandas as pd
import uproot
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()


points = pd.read_excel('../PYTHON_graphs/DATA/Energycalibration_2020-06-14.xlsx',
                  sheet_name = 'with_stretch',
                  header = 22,
                  nrows = 9,
                  usecols = "C, D",
                  convert_float = True)
                #   dtype={"B" : float, 'tof_rand': float, 'z_rand': float, 'mom_mag_rand': float,'cos(theta)_tof_rand': float})
points.dropna(inplace=True);

In [ ]:
points

In [ ]:
#polynomial fit with degree = 2
model = np.poly1d(np.polyfit(points["ph energy"], points["radius square"], 2))

#FIX THE DETERMINATION COEFFICEINT
# https://stackoverflow.com/questions/893657/how-do-i-calculate-r-squared-using-python-and-numpy
r2 = np.corrcoef(points["ph energy"], points["radius square"])[0,1]**2

#add fitted polynomial line to scatterplot
polyline = np.linspace(249, 261, 50)

fit_info = (f"y = {model[0]:.3f}x2 + {model[1]:.3f}x + {model[2]:.3f} \nR2 = {r2:.3f}")

fig,ax= plt.subplots(1,figsize=(15, 10))
# plt.suptitle("fish")
ax.scatter(points["ph energy"], points["radius square"])
ax.plot(polyline, model(polyline), "--", alpha=0.5)
ax.set_xlabel('photoelectron energy [eV]')
ax.set_ylabel('squared radius [mm2]')
ax.legend(title=fit_info, loc="best")

# fig.savefig("../PYTHON_graphs/OUTPUTS/lens_effect_fish.svg", dpi=144, transparent=False)
# fig.savefig("../PYTHON_graphs/OUTPUTS/lens_effect_fish.png", dpi=144, transparent=False)

In [ ]:
Ar = uproot.open(r"../PYTHON_graphs/DATA/Experiments/Ar_FULL_analysis.root")
Ar["all_toghether/"].keys()
Ar["all_toghether/fish filet y"].all_members
Ar["all_toghether/position"].values()


In [ ]:
fish_fillet_x=np.array(Ar["all_toghether/fish filet x"].to_numpy(),dtype=object)
fish_fillet_y=np.array(Ar["all_toghether/fish filet y"].to_numpy(),dtype=object)
cthetaEE=np.array(Ar["all_toghether/check_cthetaEE"].to_numpy(),dtype=object)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.scatter(fish_fillet_y[0][0],fish_fillet_y[0][1], cmap=cmap_temp)
ax.set_title("MFPAD tot ")

# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('normalised counts')
# ax.set_aspect(aspect)

# fig.savefig("../PYTHON_graphs/OUTPUTS/total_MFPAD_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)